In [56]:
import os
import shutil
from sklearn.model_selection import train_test_split
import torch
import torchvision.transforms as T
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import torchvision.transforms.functional as F
import random
from PIL import ImageChops
import math

### Loading Configuration

In the following steps, we will load the configuration settings using the `load_configuration` function. The configuration is stored in the `config` variable which will b

In [57]:
from config.load_configuration import load_configuration
config = load_configuration()

PC Name: DESKTOP-BLKLC13
Loaded configuration from config/config_mika.yaml


Seeds oder ähnliches Setzten, um wiederholbarkeit herzustellen??
Bei den Splits habe ich seeds händisch gesetzt

In [46]:
# Paths
source_dir = config['path_to_raw_pics']
output_dir = config['path_to_split_aug_pics']
class_names = os.listdir(source_dir)  # e.g., ['bunny', 'cat', 'dog']

# Split ratios
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# Make folders
for split in ['train', 'val', 'test']:
    os.makedirs(os.path.join(output_dir, split), exist_ok=True)

# Split and move files (no subfolders)
for class_name in class_names:
    class_dir = os.path.join(source_dir, class_name)
    files = os.listdir(class_dir)

    train_val, test = train_test_split(files, test_size=test_ratio, random_state=config['seed'])
    train, val = train_test_split(train_val, test_size=val_ratio / (train_ratio + val_ratio), random_state=config['seed'])

    for split, split_files in zip(['train', 'val', 'test'], [train, val, test]):
        for file in split_files:
            src = os.path.join(class_dir, file)

            # Prefix with class name to prevent collisions
            filename = f"{file}"
            dst = os.path.join(output_dir, split, filename)

            shutil.copy2(src, dst)

KeyError: 'path_to_raw_pics'

### Augementation

In [42]:
brightness = random.uniform(0, 0.3)
contrast = random.uniform(0, 0.3)
hue = random.uniform(0, 0.1)
saturation = random.uniform(0, 0.3)
angle = 10

# Define transforms
flip = T.RandomHorizontalFlip(p=1.0)


def rotate_crop_borders(img, angle):
    rotate_plus = T.RandomRotation(degrees=(angle, angle))
    img = rotate_plus(img)


    angle_deg = angle % 180
    if angle_deg > 90:
        angle_deg = 180 - angle_deg
    angle_rad = math.radians(angle_deg)  # Only convert once here

    sin_a = math.sin(angle_rad)
    cos_a = math.cos(angle_rad)

    if img.height * sin_a <= img.width * cos_a:
        new_w = (img.width * cos_a) - (img.height * sin_a)
        new_h = (img.height * cos_a) - (img.width * sin_a)
    else:
        new_w = (img.height * cos_a) - (img.width * sin_a)
        new_h = (img.width * cos_a) - (img.height * sin_a)

    left = (img.width - new_w) // 2
    top = (img.height - new_h) // 2
    right = left + new_w
    bottom = top + new_h
    return img.crop((left, top, right, bottom))



def apply_extra_augments(img):
    # Randomize jitter parameters
    brightness = random.uniform(0, 0.3)
    contrast = random.uniform(0, 0.3)
    hue = random.uniform(0, 0.1)
    saturation = random.uniform(0, 0.3)

    # Apply jitter and blur
    jitter = T.ColorJitter(brightness=brightness, contrast=contrast)
    hue_sat = T.ColorJitter(hue=hue, saturation=saturation)

    img = jitter(img)
    img = hue_sat(img)

    if random.random() < 0.7:  # Optional: Blur sometimes
        img = T.GaussianBlur(kernel_size=5)(img)

    return img

# Valid image extensions
valid_exts = ('.jpg', '.jpeg', '.png')

# Loop through class folders
for class_name in os.listdir(output_dir):
    class_path = os.path.join(output_dir, class_name)
    if not os.path.isdir(class_path):
        continue

    for filename in os.listdir(class_path):
        if not filename.lower().endswith(valid_exts):
            continue

        filepath = os.path.join(class_path, filename)
        name, ext = os.path.splitext(filename)

        # Load original image
        original = Image.open(filepath).convert("RGB")

        # Detect and preserve original label suffix (_ok or _nok)
        if '_ok' in name:
            base_name = name.split('_ok')[0] + '_ok'
        elif '_nok' in name:
            base_name = name.split('_nok')[0] + '_nok'
        else:
            base_name = name  # fallback if no label is found

        # v0 - original, saved as _v0
        v0_path = os.path.join(class_path, f"{base_name}_v0{ext}")
        original.save(v0_path)

        # Optionally remove the original if it’s not already a _v0 version
        if not name.endswith('_v0'):
            try:
                os.remove(filepath)
                print(f"Deleted original: {filename}")
            except Exception as e:
                print(f"Could not delete original {filename}: {e}")


        # v1 - flipped
        v1 = flip(original)
        v1.save(os.path.join(class_path, f"{base_name}_v1{ext}"))

        # v2 - rotated +5° + jitter + blur
        v2 = rotate_crop_borders(original, angle)
        v2 = apply_extra_augments(v2)
        v2.save(os.path.join(class_path, f"{base_name}_v2{ext}"))

        # v3 - rotated -5° + jitter + blur
        v3 = rotate_crop_borders(original, angle*-1)
        v3 = apply_extra_augments(v3)
        v3.save(os.path.join(class_path, f"{base_name}_v3{ext}"))

        # v4 - flipped + rotated +5° + jitter + blur
        v4 = rotate_crop_borders(original, angle)
        v4 = apply_extra_augments(v4)
        v4.save(os.path.join(class_path, f"{base_name}_v4{ext}"))

        # v5 - flipped + rotated -5° + jitter + blur
        v5 = rotate_crop_borders(original, angle*-1)
        v5 = apply_extra_augments(v5)
        v5.save(os.path.join(class_path, f"{base_name}_v5{ext}"))

        print(f"Generated 6 variants for: {filename}")

Deleted original: Frettchen_0002_nok.jpg
Generated 6 variants for: Frettchen_0002_nok.jpg
Deleted original: Zwergkaninchen_0002_ok.jpg
Generated 6 variants for: Zwergkaninchen_0002_ok.jpg
Deleted original: Zwergkaninchen_0009_ok.jpg
Generated 6 variants for: Zwergkaninchen_0009_ok.jpg
Deleted original: Frettchen_0001_nok.jpg
Generated 6 variants for: Frettchen_0001_nok.jpg
Deleted original: Frettchen_0003_nok.jpg
Generated 6 variants for: Frettchen_0003_nok.jpg
Deleted original: Zwergkaninchen_0003_ok.jpg
Generated 6 variants for: Zwergkaninchen_0003_ok.jpg
Deleted original: Zwergkaninchen_0004_ok.jpg
Generated 6 variants for: Zwergkaninchen_0004_ok.jpg
Deleted original: Zwergkaninchen_0006_ok.jpg
Generated 6 variants for: Zwergkaninchen_0006_ok.jpg
Deleted original: Zwergkaninchen_0007_ok.jpg
Generated 6 variants for: Zwergkaninchen_0007_ok.jpg
Deleted original: Zwergkaninchen_0008_ok.jpg
Generated 6 variants for: Zwergkaninchen_0008_ok.jpg
Deleted original: Zwergkaninchen_0010_ok.jpg